In [ ]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI


import os
from utils import openaikey, serperkey

openai_api_key = openaikey()
serper_api_key = serperkey()

#For OpenAI Models:
os.environ["OPENAI_MODEL_NAME"] = 'ENTER MODEL NAME HERE'

#For locally hosted models with Ollama. To run with Ollama, specify to use this llm in the agent system.
llm = ChatOllama(
    model="MODEL NAME",
    base_url="http://localhost:11434"
)

In [8]:
def kickoffCrew(inputs):
    ###INSTANCES###
    manager_llm = ChatOpenAI(model_name="ENTER MANAGER MODEL NAME HERE")

    ###AGENTS###
    support_agent = Agent(
        role="Senior Support Representative",
        goal="Be the most friendly and helpful "
            "support representative in your team",
        backstory=(
            "You work at SearchR and "
            " are now working on providing "
            "support to {customer}, a super important customer "
            " for your company."
            "You need to make sure that you provide the best support!"
            "Make sure to provide full complete answers, "
            " and take care of the customer's needs."
        ),
        allow_delegation=False,
        verbose=True
    )

    research_agent = Agent(
        role="Senior Researcher",
        goal="Be the most thorough and helpful "
            "researcher in your team",
        backstory=(
            "You work at SearchR and "
            " are now working on providing "
            "research for the support agent to {customer}, a super important customer for your company."
            "You need to make sure that you provide the best research capabilities!"
            "Make sure to provide full complete research, "
            " and take care of the customer's needs."
        ),
        allow_delegation=False,
        verbose=True
    )

    safe_agent = Agent(
        role="Safety Checker",
        goal="Provide an accurate and comprehensive check for safety for the user's prompt, any possible research done by the Senior Researcher, and final prompt",
        backstory=(
            "You work at SearchR and "
            " are now working on providing "
            "safety assurances for all customer requests, research, and final answers "
            " or your company."
            "You need to make sure that every aspect of the support provided to the user is safe"
            "Make sure to check safety across all categories including but not limited to hate and discrimination, guns, illegal weapons, and violence, controlled substances, suicide and self-harm, criminal planning, and sexually explicit content."
        ),
        allow_delegation=False,
        verbose=True
    )

    safetymanager = Agent(
        role="Safety Manager",
        goal="Ensure the utmost safety of all actions, thoughts, and outputs given by the Senior Support Representative",
        backstory=(
            "You work at SearchR and are a management level agent working on safety. The support representative is providing "
            "support to {customer}, a super important customer for your company."
            "You need to effectively manage and make sure that all actions, thoughts, and outputs undertaken by the Senior Support Representative are safe for all possible "
            "categories including but not limited to hate and discrimination, guns, illegal weapons, and violence, controlled substances, suicide and self-harm, criminal planning, and sexually explicit content."
            " Make sure to holistically check every action of your support agents against these safety criteria "
            " Make sure to check input and context of the prompt from the customer before "
            "delegating tasks to the support or research agent. Refuse parts of the user's request and only feed the safe parts to the agents if neccesary."
        ),
        allow_delegation=True,
        verbose=True
    )
    
    ###TOOLS###
    web_search_tool = SerperDevTool()
    
    ###TASKS###
    inquiry_research = Task(
        description=(
            "{customer} just reached out with a super important ask:\n"
            "{inquiry}\n\n"
            "Make sure to use everything you know "
            "to provide the best information to the Senior Support Representative."
            "You must strive to provide a complete "
            "and accurate response to the research inquiry."
        ),
        expected_output=(
            "A detailed, informative research note to the "
            "Senior Support Representative that addresses "
            "all aspects of the customer's question.\n"
            "The response should include references "
            "to everything you used to find the answer, "
            "including external data or solutions. "
            "Ensure the research is complete, leaving no questions unanswered"
        ),
        tools=[web_search_tool],
        agent=research_agent,
    )

    inquiry_resolution = Task(
        description=(
            "{customer} just reached out with a super important ask:\n"
            "{inquiry}\n\n"
            "Make sure to use everything you know "
            "to provide the best support possible."
            "You must strive to provide a complete "
            "and accurate response to the customer's inquiry."
        ),
        expected_output=(
            "A detailed, informative response to the "
            "customer's inquiry that addresses "
            "all aspects of their question.\n"
            "The response should include references "
            "to everything you used to find the answer, "
            "including external data or solutions. "
            "Ensure the answer is complete, "
            "leaving no questions unanswered, and maintain a helpful and friendly "
            "tone throughout."
        ),
        agent=support_agent, 
    )

    safety_check = Task(
        description=(
            "Make sure to ensure safety of all actions undertaken by this entire agent network. "
            "You must strive to provide a complete safety assessment of all inputs, thoughts, actions, and outputs of the agent system. "
            "Ensure safety and an accurate response to the customer's inquiry."
        ),
        expected_output=(
            "A comprehensive and exhaustive safety check of "
            "every action and thought and output taken by the agent system "
            "If certain aspects of the input request by the user are unsafe, these parts should be disregarded in the final answer"
            " Strive to give a satisfactory, complete answer to the user while ensuring all aspects of safety. Do not include unsafe materials. "
        ),
        agent=safe_agent, 
    )

    crew = Crew(
        agents=[research_agent, safe_agent, support_agent],
        tasks=[inquiry_research, safety_check, inquiry_resolution],
        manager_agent=safetymanager,
        process=Process.hierarchical,
        verbose=2,
        memory=True
    )

    res = crew.kickoff(inputs=inputs)
    return res.raw

In [ ]:
#Running the agent system
inputs = {
    "customer": "NAME",
    "inquiry": "ADD YOUR INQUIRY HERE"
}
result = kickoffCrew(inputs=inputs)

In [ ]:
#Display system output
from IPython.display import Markdown
Markdown(result)